# Clase 7: Pandas II

**MDS7202: Laboratorio de Programación Científica para Ciencia de Datos**

## Objetivos de la Clase

- Comprender como fusionar distintas fuentes de datos a partir de `concatenaciones` y `merge`.
- Reorganizar los datos usando `transpose`, `pivot` y `melt`
- Trabajar con dataframes con `multi-índices`
- Operar en Pandas con diferentes tipos de datos: `strings`, `fechas`, `categorías` y `ordinales`


In [ ]:
import pandas as pd

bli_df = pd.read_excel("../../recursos/2023-02/pandas-2/dataset.xlsx", header=1, index_col=0)
bli_df.head()

In [ ]:
bli_df.shape

In [ ]:
temp_df = pd.read_csv(
    "../../recursos/2023-02/pandas-2/temperature.csv")
temp_df.head(20)

In [ ]:
temp_df.shape

----

## 1. Concatenación


> Según Wikipedia: *Es la operación por la cual dos caracteres se unen para formar una cadena de caracteres (o string). También se pueden concatenar dos cadenas de caracteres o un carácter con una cadena para formar una cadena de mayor tamaño*. Ejemplo: 

In [ ]:
a = "Hola "

b = "a todos 🤗"


a + b

La idea general de concatenar es unir 2 o más `Dataframes` por filas o columnas.

<div align='center'>
    <img src='https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2023-01/10-Pandas3/concat.png' width=900/>
</div>


Todas las operaciones se hacen a través de la operación sobre los índices de los `DataFrames`.

### Caso 1: Concatenar Filas

En el caso de contactenar por filas `(axis=0)`, los `DataFrames` se unen al final a través de los índices.

<div align='center'>
    <img src='https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2023-01/10-Pandas3/merging_concat_basic.png' width=500/>
</div>

In [ ]:
bli_df = bli_df.reset_index()

Supongamos que tenemos 3 dataframes distintos:

In [ ]:
# 1 dataframe por continente: sudamerica, norteamerica, oceania
sudamerica_df = bli_df.loc[bli_df["Country"].isin(["Chile","Brazil","Colombia",]), :]
norteamerica_df = bli_df.loc[bli_df["Country"].isin(["Canada", "United States", "Mexico",]), :]
oceania_df = bli_df.loc[bli_df["Country"].isin(["New Zealand", "Australia"]), :]

Para ejecutar la concatenación, usamos el método `pd.concat` sobre una lista con los `DataFrames` por concatenar.

In [ ]:
df_concatenado_filas = pd.concat(
    [sudamerica_df, norteamerica_df, oceania_df], 
    axis=0) # para concatenar verticalmente
df_concatenado_filas

### Caso 2: Concatenar Columnas

En este caso, los `DataFrames` se unen por los índices y las columnas.


<div align='center'>
    <img src='https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2023-01/10-Pandas3/merging_concat_mixed_ndim.png' width=700/>
</div>

In [ ]:
env_df = bli_df.loc[:, ["Country", "Air pollution", "Water quality"]]
env_df.head()

In [ ]:
health_df = bli_df.loc[
    :, [
        "Country", 
        "Self-reported health", 
        "Life expectancy", 
        "Life satisfaction",
    ]
]

health_df.head()

In [ ]:
env_health_df = pd.concat(
    [env_df, health_df],
    axis=1 # concatenar horizontalmente
)

env_health_df.head()

In [ ]:
env_health_df.loc[:, "Country"].head()

Nota: La unión sigue siendo por filas. Por ende, una columna repetida aparecerá dos veces en el `DataFrame` resultante, como en el caso anterior con `Country`

> **Nota**: Para facilitar la práctica, solo dejaremos una columna `Country`

In [ ]:
env_health_df = env_health_df.loc[:, ~env_health_df.columns.duplicated()]
env_health_df.head()

### Un `DataFrame` tiene menos datos que el otro

En este caso, rellena los valores de las filas sin valor con `np.nan`.

<div align='center'>
    <img src='https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2023-01/10-Pandas3/merging_concat_axis1.png' width=800/>
</div>

In [ ]:
env_df_reducido = env_df[0:7]
env_df_reducido

In [ ]:
health_df[:15]

In [ ]:
pd.concat(
    [
        env_df_reducido, 
        health_df[:15]
    ], axis=1)

---

## 2. Merge

Es una forma de combinar dos `DataFrames` en la que usamos los valores de columna como identificador comunes para concatenar el resto de los valores:

![Idea del Merge](https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2023-01/10-Pandas3/merge.png)


Es equivalente a las sentencias `JOIN` de SQL. Existen varios tipos. `Pandas` implementa 5 a través de la función `pd.merge`.

En los siguientes ejemplos uniremos los datasets de la OECD y de temperatura agregada según los distintos tipos de `Merge`. Será de mucha utilidad pensar los `Merge` como operaciones sobre conjuntos.

In [ ]:
temp_df = pd.read_csv("../../recursos/2023-02/pandas-2/temperature.csv")
t_agg_df = temp_df.groupby("Country").agg({"Temperature": ["median", "min", "max"]})
t_agg_df = t_agg_df.droplevel(0, axis=1).reset_index()
t_agg_df.columns = ["Country", "t_median", "t_min", "t_max"]
t_agg_df.head()

---

In [ ]:
env_health_df.head()

### Inner

Combina los elementos que se encuentren en ambas tablas. Descarta todo el resto

![Inner](https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2023-01/10-Pandas3/inner.png)

In [ ]:
inner_merge_df = pd.merge(
    left=env_health_df,
    right=t_agg_df,
    left_on="Country",
    right_on="Country",
    how="inner",
    sort=True,
)
inner_merge_df.head()

In [ ]:
inner_merge_df.shape

### Left Merge

![Right Merge](https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2023-01/10-Pandas3/left.png)

Conserva solo los elementos que se hayan combinado correctamente provenientes dataset `left`.

In [ ]:
left_merge_df = pd.merge(
    left=env_health_df,
    right=t_agg_df,
    on="Country",
    how="left",
    sort=True,
)

left_merge_df.head()

In [ ]:
left_merge_df.shape

### Right Merge

Conserva solo los elementos que se hayan combinado correctamente provenientes dataset `right`.

![Right Merge](https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2023-01/10-Pandas3/right.png)

In [ ]:
right_merge_df = pd.merge(
    left=env_health_df,
    right=t_agg_df,
    on="Country",
    how="right",
    sort=True,
)

right_merge_df.head()

In [ ]:
right_merge_df.shape

### Outer

Combina todos los elementos posibles y conserva todo el resto en filas independientes.

![Outer Join](https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2023-01/10-Pandas3/outer.png)

In [ ]:
outer_merge_df = pd.merge(
    left=env_health_df,
    right=t_agg_df,
    on="Country",
    sort=True,
    how="outer",
)
outer_merge_df

In [ ]:
outer_merge_df.shape

#### Con Indicador

In [ ]:
outer_merged_df = pd.merge(
    left=env_health_df,
    right=t_agg_df,
    on="Country",
    how="outer",
    sort=True,
    indicator=True,
)
outer_merged_df.head()

In [ ]:
outer_merged_df[outer_merged_df["_merge"] == "left_only"].head()

In [ ]:
outer_merged_df[outer_merged_df["_merge"] == "both"].head()

In [ ]:
outer_merged_df[outer_merged_df["_merge"] == "right_only"].head()

---

## 3. Transponer Datos

Simplemente invertir las filas por las columnas.

In [ ]:
bli_df.head()

In [ ]:
bli_df.T.head()

---

## 4. Pivot y Melt

### Introducción

El dataset que usamos la clase pasada está relativamente ordenado.

In [ ]:
bli_df.head(5).iloc[:, 0:20]

In [ ]:
bli_df.loc[:, ["Country", "Labour market insecurity"]].head()

Sin embargo, originalmente tenía la siguiente estructura:

In [ ]:
dataset_original = pd.read_csv("../../recursos/2023-02/pandas-3/bli_original.csv", keep_default_na=False)
dataset_original.head()

In [ ]:
dataset_original.loc[:, ["Country", "Indicator", "Value"]].sort_values('Country').tail(10)

In [ ]:
dataset_original.shape

Cada fila de este dataset contiene información acerca de los paises y de los indicadores y el valor del indicador. Esta forma es conocida como **long**. 

### Pivot

![Pivot](../../recursos/2024-01/pandas-2/pivot.png)

Para convertirla al formato con el que hemos estado trabajando, **wide**, debemos **pivotear** la tabla:

In [ ]:
dataset_original.head(3)

In [ ]:
dataset_original.head().loc[:, ["Country", "Indicator", "Value"]] # comenzamos desde formato long

> **Ejercicio ✏️**: Pivotear la tabla original de los datos de la OECD

In [ ]:
# transformamos a formato wide
pd.pivot_table(
    dataset_original, 
    index='Country', 
    columns='Indicator', 
    values='Value'
).head(5)

### Melt

Y como transformamos un dataframe **wide** a **long**? Podemos usar la operación `melt`:

![Melt](../../recursos/2023-02/pandas-4/melt.png)

In [ ]:
# partimos desde el dataframe con formato wide
bli_df.head()

In [ ]:
# transformamos a formato long
bli_df.melt(id_vars = 'Country', # variable id sobre la cual hacer unpivot
            var_name = 'Indicator', # nombre de la columna "variable"
            value_name = 'Value') # nombre de la columna "valor"

Usando su argumento `value_vars` podemos seleccionar solo alguna de las columnas que deseamos operar con `melt`.
De todas formas, este comportamiento también puede ser logrado usando un simple indexador `.loc`

---

## 5. Multi-Índices

Hasta el momento solo hemos trabajado con `Dataframes` que contienen solo un nivel de filas o columnas. Sin embargo, es posible también agregar **más niveles** a los indices y a las columnas. 
Esto se le conoce como **<u>multi-índice</u>**.

In [ ]:
dataset_original.loc[:, 
                     ["Continent", 
                      "Country", 
                      "Indicator", 
                      "Unit", 
                      "Value"]].head()

Para agregar niveles de columnas, en el proceso de pivoteo vamos a indicar que tanto `Unit` como `Indicator` sean niveles de las columnas; y que a la vez, tanto `Continent` como `Country` sean indices para las filas. 

El resultado de esto puede ser visto en el siguiente `DataFrame`:

In [ ]:
# primero seteamos unas opciones de display de pandas
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

In [ ]:
# generamos dataframe con multi-índices usando pivot
dataset_multindex = pd.pivot_table(
    dataset_original,
    index=["Continent", "Country"],
    columns=["Unit", "Indicator"],
    values="Value",
)
dataset_multindex.head()

In [ ]:
dataset_multindex.index

In [ ]:
dataset_multindex.columns.values

### Acceder a Multi-Índices

In [ ]:
# Seleccionar la fila que contiene a Chile
dataset_multindex.loc[[('SA', 'Chile')], :]

In [ ]:
# Seleccionar las columnas de los indicadores basados en Porcentajes
dataset_multindex.loc[:, ['Percentage']]

### `droplevel`

El método `droplevel` nos permite eliminar un nivel de un multi-índice, tanto para filas como para columnas.
Recibe como parámetros el nivel (partiendo por 0 desde afuera hacia adentro) y el eje (axis): 

In [ ]:
dataset_multindex.head()

In [ ]:
dataset_multindex.droplevel(0, axis=0).head() # dropear nivel 0 de las filas

In [ ]:
dataset_multindex.droplevel(1, axis=0).head() # dropear nivel 1 de las filas

In [ ]:
dataset_multindex.droplevel(0, axis=1).head() # dropear nivel 0 de las columnas

---

## 6. Strings en Pandas

La mayoría de los métodos de procesamiento de strings que se vieron anteriormente pueden ser ejecutados a través del atributo `.str` en las series de Pandas.

La idea detrás del uso de los métodos de `.str` es que el preprocesamiento se haga de manera ordenada y eficiente. Al utilizar estos métodos, los usuarios pueden aplicar una variedad de transformaciones a sus datos de texto en una sola línea de código, lo que facilita el procesamiento de grandes conjuntos de datos de texto.

In [ ]:
mascotas = [
    [
        "Perro",
        "el perro (Canis familiaris o Canis lupus familiaris, dependiendo de si "
        "se lo considera una especie por derecho propio o una subespecie del "
        "lobo),1​2​3​ llamado perro doméstico o can,4​ y en algunos lugares"
        " coloquialmente llamado chucho,5​ tuso,6​ choco,7​ entre otros; es un "
        "mamífero carnívoro de la familia de los cánidos, que constituye "
        "una especie del género Canis.8​9​. Posee un oído y un olfato muy "
        "desarrollados, y este último es su principal órgano sensorial.  \n"
        ],
    [
        "Gato",
        "el gato doméstico1​2​ (Felis silvestris catus), llamado popularmente "
        "gato, y de forma coloquial minino,3​ michino,4​ michi,5​ micho,"
        "6​ mizo,7​ miz,8​ morroño9​ o morrongo,10​ entre otros nombres, es "
        "un mamífero carnívoro de la familia Felidae. Es una subespecie "
        "domesticada por la convivencia con el ser humano.  \n"
    ],
    [
        "Canario",
        "el canario doméstico (Serinus canaria domestica)3​4​ es una "
        "subespecie desarrollada durante siglos de selección en cautividad "
        "partiendo de ejemplares del canario silvestre o canario salvaje "
        "(Serinus canaria), una especie de ave del orden paseriforme de "
        "la familia de los fringílidos, endémica de las islas Canarias, "
        "Azores y Madeira.5​6​   \n"
    ],
]

df_mascotas = pd.DataFrame(mascotas, columns=["nombre", "resumen"])
df_mascotas

### Len, Lower, Upper, Title, Capitalize

In [ ]:
df_mascotas.loc[:, "resumen"].str

In [ ]:
df_mascotas.loc[:, "resumen"].str.len()

In [ ]:
df_mascotas.loc[:, "resumen"].str.lower()

In [ ]:
df_mascotas.loc[:, "resumen"].str.upper()

In [ ]:
df_mascotas.loc[:, "resumen"].str.title()

In [ ]:
df_mascotas.loc[:, "resumen"].str.capitalize()

### Contains, Split, Join, Replace

In [ ]:
df_mascotas.loc[:, "resumen"].str.contains("perro")

In [ ]:
df_mascotas.loc[:, "resumen"].str.contains("gato")

In [ ]:
df_mascotas.loc[:, "resumen"].str.split(" ")

In [ ]:
df_mascotas.loc[:, "resumen"].str.split(" ").str.join(" ")

In [ ]:
df_mascotas.loc[:, "resumen"].values

In [ ]:
s1 = (
    df_mascotas.loc[:, "resumen"]
    .str.replace("(", "", regex=False)
    .str.replace(")", "", regex=False)
    .str.replace("perro", "perrito", regex=False)
    .str.replace("gato", "gatito", regex=False)
)

s1

> **Pregunta ❓:** ¿Qué indica el parámetro `regex`?

### Otra forma de hacer lo anterior? Método `apply`

In [ ]:
df_mascotas.loc[:, "resumen"].apply(lambda x: len(x))

In [ ]:
df_mascotas.loc[:, "resumen"].apply(lambda x: x.lower())

In [ ]:
df_mascotas.loc[:, "resumen"].apply(lambda x: x.split(' '))

> **Ejercicio ✏️**: Realizar el resto de las operaciones de `.str` usando `.apply`

---

## 7. Fechas

Volveremos a usar el dataframe de temperaturas!

Las fechas y horarios son por lo general parte íntegra de los datos en muchas aplicaciones. Algunos ejemplos de uso de fechas y horarios en aplicaciones son el análisis de datos de ventas a lo largo del tiempo, el seguimiento de la evolución del clima, el análisis de la actividad del usuario en una plataforma digital, entre otros.

> **Pregunta ❓**: ¿Qué aplicación particular podríamos darle al manejar los datetimes del clima? 

In [ ]:
temp_df.head()

### Módulo `Datetime`

El módulo datetime en Python es una librería estándar (built-in) que proporciona clases y métodos para trabajar con fechas y horas los cuales simplifican la manipulación y cálculos ellos.

In [ ]:
import datetime

#### Date

Objeto que almacena día, mes y año.

In [ ]:
date_object = datetime.date.today()
print(date_object)

In [ ]:
date_object.day

In [ ]:
date_object.month

In [ ]:
date_object.year

#### Datetime

Almacena segundos, minutos, hora, día, mes y año. También puede contener timezone.

In [ ]:
datetime_object = datetime.datetime.now()
print(datetime_object)

In [ ]:
datetime_object.second

In [ ]:
datetime_object.day

#### Instanciar nuevos Date y Datetimes

In [ ]:
d = datetime.date(2021, 9, 9)
print(d)

In [ ]:
print(datetime.date)

In [ ]:
print("Año:", d.year)
print("Mes:", d.month)
print("Día:", d.day)

In [ ]:
d = datetime.datetime(
    year=2021, 
    month=4, 
    day=19, 
    hour=10, 
    minute=59, 
    second=55
)
print(d)

In [ ]:
print("Hora:", d.hour)
print("Minuto:", d.minute)
print("Segundo:", d.second)
print("Microsegundo:", d.microsecond)

> **Pregunta ❓**: ¿Podemos sumar o restar fechas?

In [ ]:
datetime.datetime.now() + datetime.datetime(1, 1, 1)

#### TimeDelta

Podemos realizar operaciones entre fechas mediante `timedelta`

In [ ]:
d = datetime.datetime.now()
d

In [ ]:
from datetime import timedelta

t1 = timedelta(weeks=2)
t1

In [ ]:
d + t1

In [ ]:
d - t1

###  Datos temporales en Pandas

Pandas implementa su propio sistema de datetimes.
`pd.to_datetimes` nos permite convertir una `Serie` o un `DataFrame` en una `Serie` de datetimes.

In [ ]:
temp_df

In [ ]:
dates = temp_df.loc[:, ["Year", "Month"]].copy() # seleccionar columnas de fecha
dates.loc[:, "Day"] = 1 # crear columna "Dia"

dates = dates.astype(str)
concat_dates = (
    dates.loc[:, 'Year']
    + ' '
    +  dates.loc[:, 'Month'] 
    + ' '
    + dates.loc[:, 'Day']
)

concat_dates # columna str

In [ ]:
# transformar a datetime
parsed_dates = pd.to_datetime(concat_dates)

# creamos esta columna en el dataframe
temp_df.loc[:, 'dates'] = parsed_dates
temp_df

#### Indexado

Podemos fijar las fechas como índices y luego indexar por rangos de estas

In [ ]:
temperaturas_por_fecha = temp_df.set_index("dates")
temperaturas_por_fecha

In [ ]:
# Rango 1995-1-1 al 199-12-1
temperaturas_por_fecha.loc["1995-1-1":"1999-12-1"]

In [ ]:
temperaturas_por_fecha.loc["1999-11-1":"1999-12-1"]

#### Timedeltas

Podemos también sumar/restar unidades de tiempo a columnas `datetime` en Pandas

In [ ]:
ptd1 = pd.Timedelta(weeks=2)
ptd1

In [ ]:
temp_df.loc[:, "dates"] + ptd1

---

##  8. Categorías

Una variable categórica es un tipo de dato que puede tomar un número limitado (y usualmente fijo) de posibles valores.
Ejemplos de esto: Género, clase social, tipo de sangre, etc...

En general, guardar los datos como categóricos es mucho más eficiente que guardarlos como string. Según la referencia de pandas:

> The memory usage of a Categorical is proportional to the number of categories plus the length of the data. In contrast, an object dtype is a constant times the length of the data.

Como ejemplo, usaremos los continentes a los que pertenece cada país:

In [ ]:
countries = pd.read_csv(
    "https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2023-01/11-Pandas4/country-and-continent.csv"
)
countries = countries.loc[:, ["Continent_Name", "Country_Name"]]
countries

Declaramos una variable como categórica transformando la serie a `category` con `.astype("category")`:

In [ ]:
countries["Continent_Name"]

In [ ]:
countries["Continent_Name"] = countries.loc[:, "Continent_Name"].astype("category")
countries["Continent_Name"]

Podemos acceder a las categorías con:

In [ ]:
countries["Continent_Name"].cat.categories

Podemos renombrar categorias de la siguiente manera:

In [ ]:
countries["Continent_Name"] = countries["Continent_Name"].cat.rename_categories(
    {
        "Europe": "Europa",
    }
)

Se pueden agregar categorías

In [ ]:
countries["Continent_Name"].cat.add_categories(["Atlantida"])

Como también eliminar las no usadas:

In [ ]:
# Remover las no usadas
countries["Continent_Name"].cat.remove_unused_categories()

O incluso, eliminar una categoría completa. Noten que esto transforma valores de esa categoría a `NaN`.

In [ ]:
# Como también remover una categoría completa.
countries["Continent_Name"].cat.remove_categories(["Europa"])

---

## 9. Ordinales

Una variable ordinal es un tipo de variable categórica que representa una característica o atributo que se puede **ordenar** en diferentes niveles categorías bajo un orden predefinido.

Por ejemplo, hagamos una clasificación muy simple del clima a partir de la temperatura media. Para esto, calculemos quintiles:

> Nota: Los elementos meteorológicos a tomar en cuenta para definir un clima son la temperatura, la presión, el viento, la humedad y la precipitación. Referencias: https://es.wikipedia.org/wiki/Clima

In [ ]:
# Preparemos el dataset

temp_agg = temp_df.groupby("Country").agg({"Temperature": ["mean", "std"]})

temp_agg.columns = temp_agg.columns.droplevel()
temp_agg = temp_agg.reset_index()
temp_agg.columns = ["Country", "Mean temperature", "Std temperature"]
temp_agg

merged_df = pd.merge(
    temp_agg,
    countries,
    how="left",
    left_on="Country",
    right_on="Country_Name",
)

# obtenemos la media de temperatura
mean_temp = merged_df["Mean temperature"]
mean_temp


In [ ]:
# quintiles nombrados
clima_ordinal = pd.qcut(
    mean_temp, 5, labels=["Polar", "Frio", "Templado", "Calido", "Muy Calido"]
)
clima_ordinal

In [ ]:
clima_ordinal.min()

In [ ]:
clima_ordinal.max()

In [ ]:
clima_ordinal.value_counts()

In [ ]:
clima_ordinal.mode()

In [ ]:
clima_ordinal.sort_values()